# Running a simulation from the models created from the basic example

> This notebook will run after the basic example is created. Please make sure that `00_Basic_Example.ipynb` has been run.

In [1]:
# Load the system model
import os
from MachSysS.system_structure_pb2 import MachinerySystem
from MachSysS.convert_to_feems import convert_proto_propulsion_system_to_feems

file_path = os.path.join("data", "electric_power_system.pb")
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Model file not found at {file_path}. Please run `00_Basic_Example.ipynb` first.")

# Load the protobuf file
with open(file_path, "rb") as f:
    system = MachinerySystem()
    system.ParseFromString(f.read())
# Convert the system to FEEMS format
feems_system = convert_proto_propulsion_system_to_feems(system)


In [4]:
# Run the FEEMS simulation using RunFeemsSim
from pprint import pprint
import numpy as np
import pandas as pd
from RunFeemsSim.machinery_calculation import MachineryCalculation
from feems.fuel import FuelSpecifiedBy

machinery_calculation = MachineryCalculation(feems_system)
# Set the time series for propulsion and auxiliary power
# This example only works if the two propulsion drives have the identical power load.
number_points_load = 1000
time_interval_s = 1.0  # seconds
rated_power_for_propulsion_drive_1 = feems_system.propulsion_drives[0].rated_power
rated_power_for_propulsion_drive_2 = feems_system.propulsion_drives[1].rated_power
propulsion_power_ts = pd.DataFrame(
    index=np.arange(0, number_points_load * time_interval_s, time_interval_s),
    data={
        feems_system.propulsion_drives[0].name : rated_power_for_propulsion_drive_1 * np.random.random(number_points_load),
        feems_system.propulsion_drives[1].name: rated_power_for_propulsion_drive_2 * np.random.random(number_points_load),
    }
)
auxiliary_load = 100 * np.random.random(number_points_load)

# Calculate the machinery system output from the propulsion power time series
result = machinery_calculation.calculate_machinery_system_output_from_propulsion_power_time_series(
    propulsion_power=propulsion_power_ts,
    auxiliary_power_kw=auxiliary_load,
    fuel_specified_by=FuelSpecifiedBy.FUEL_EU_MARITIME,
)

# Print the results
pprint(result.__dict__)


{'co2_emission_total_kg': GHGEmissions(tank_to_wake_kg_or_gco2eq_per_gfuel=np.float64(159.8000914094511),
                                       well_to_tank_kg_or_gco2eq_per_gfuel=np.float64(30.172940867573164),
                                       tank_to_wake_kg_or_gco2eq_per_gfuel_without_slip=np.float64(157.32248312099847),
                                       tank_to_wake_kg_or_gco2eq_per_gfuel_from_green_fuel=np.float64(0.0),
                                       tank_to_wake_kg_or_gco2eq_per_gfuel_without_slip_from_green_fuel=np.float64(0.0)),
 'detail_result':                                 multi fuel consumption [kg]  \
Genset 1  FuelConsumption(fuels=[<feems.fuel.Fuel object...   
Genset 2  FuelConsumption(fuels=[<feems.fuel.Fuel object...   

         electric energy consumption [MJ] mechanical energy consumption [MJ]  \
Genset 1                                0                         348.889167   
Genset 2                                0                         562

In [ ]:
# Print the fuel consumption
print("\nFuel Consumption Results:")
for fuel_consumption in result.multi_fuel_consumption_total_kg.fuels:
    print(f"{fuel_consumption.fuel_type.name}: {fuel_consumption.mass_or_mass_fraction:.2f} kg")
# Print the ghg emissions
print("\nGHG Emissions Results:")
print(f"  - well to tank: {result.co2_emission_total_kg.well_to_tank_kg_or_gco2eq_per_gfuel:.2f} kg")
print(f"  - tank to wake: {result.co2_emission_total_kg.tank_to_wake_emissions_kg_for_ets:.2f} kg")
print(f"  - well to wake: {result.co2_emission_total_kg.well_to_wake_kg_or_gco2eq_per_gfuel:.2f} kg")
# Print the result per component
print("\nResult per component")
print(result.detail_result)


Fuel Consumption Results:
DIESEL: 49.07 kg

GHG Emissions Results:
  - well to tank: 30.17 kg
  - tank to wake: 159.80 kg
  - well to wake: 189.97 kg

Result per component
                                multi fuel consumption [kg]  \
Genset 1  FuelConsumption(fuels=[<feems.fuel.Fuel object...   
Genset 2  FuelConsumption(fuels=[<feems.fuel.Fuel object...   

         electric energy consumption [MJ] mechanical energy consumption [MJ]  \
Genset 1                                0                         348.889167   
Genset 2                                0                         562.137969   

         energy_stored [MJ] running hours [h]  \
Genset 1                  0          0.215278   
Genset 2                  0          0.234167   

                                          CO2 emission [kg] NOx emission [kg]  \
Genset 1  GHGEmissions(tank_to_wake_kg_or_gco2eq_per_gfu...          0.793116   
Genset 2  GHGEmissions(tank_to_wake_kg_or_gco2eq_per_gfu...          1.277886   

    